In [4]:
import os
import sys
from importlib import reload
from pathlib import Path

import numpy as np
from torch.nn import ReLU, Linear
from torch_geometric.nn import GCNConv

In [5]:
# NOTE: Boilerplate setup for Jupyter imports
sys.path.append(Path(os.getcwd()).parent.as_posix())

import configs as config_module
import enhancer as enhancer_module
import encoders as encoders_module
import scheme.network as network_module
import scheme.data as data_module

reload(config_module)
reload(enhancer_module)
reload(encoders_module)
reload(network_module)
reload(data_module)

PathConfig, TrainConfig = config_module.PathConfig, config_module.TrainConfig
Enhancer = enhancer_module.Enhancer
get_default_encoders = encoders_module.get_default_encoders
GNNConfig, EnhancerData = network_module.NetworkConfig, data_module.EnhancerData

In [6]:
path_config = PathConfig(
    data_dir="../../data"
)

with open(path_config.data_root / "processed/cora.npz", "rb") as f:
    unpacked = np.load(f)

    # NOTE: Target dimensions
    data = EnhancerData(
        unpacked["data"],
        unpacked["target"].reshape(-1),
        unpacked["spatial"],
    )

In [7]:
gnn_setup = GNNConfig(
    encoder=[
        GCNConv(data.features.shape[1], 256),
        GCNConv(256, 256),
    ],
    estimator=[
        Linear(256, 128),
        ReLU(),
        Linear(128, 128),
        ReLU(),
        Linear(128, len(np.unique(data.target))),
    ]
)

In [8]:
encoders = [
    encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist",
    ),
    encoders_module.ReprEncoder(
        neighbor_rate=0.7,
        cache_dir=path_config.edge_cache,
        note="cora_repr",
    ),
]

result = Enhancer.compare_builders(data, gnn_setup, encoders)

cora_dist test: Loss = 1.4003e+00 | Accuracy = 0.7121771217712177
cora_repr test: Loss = 1.4898e+00 | Accuracy = 0.4612546125461255
DistEncoder test: Loss = 1.4317e+00 | Accuracy = 0.6531365313653137
ReprEncoder test: Loss = 1.5130e+00 | Accuracy = 0.43911439114391143


In [9]:
print(result)

Option         accuracy_score    f1_score
-----------  ----------------  ----------
cora_dist            0.712177    0.659564
cora_repr            0.461255    0.365716
DistEncoder          0.653137    0.592432
ReprEncoder          0.439114    0.346177


In [10]:
enhancer = Enhancer(
    net_config=gnn_setup,
    edge_builder=encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist [max dist = 5]",
    ),
)

In [11]:
gnn = enhancer.fit(data)

In [12]:
enhancer.transform(data)

tensor([[-0.0473,  0.3497, -0.0536,  ..., -0.7110,  0.1269, -0.1221],
        [-0.2279, -0.2066, -0.1728,  ..., -0.3087, -0.2231,  0.3358],
        [-0.4179, -0.4198, -0.1217,  ..., -0.5019, -0.2199,  0.2649],
        ...,
        [-0.0457,  0.1450,  0.3061,  ..., -1.2240, -0.0355,  0.0135],
        [ 0.0207,  0.1969, -0.0421,  ..., -0.5167, -0.2202, -0.1670],
        [-0.1874,  0.0948,  0.1206,  ..., -0.5581, -0.0809,  0.0260]],
       grad_fn=<AddBackward0>)